In [1]:
import pandas as pd

In [2]:
filename = 'data/sfpd_clean.csv'
sfdata = pd.read_csv(filename)


In [3]:
from geopy.geocoders import Nominatim
import re
def get_zipcode(loc):
    geolocator = Nominatim()
    loc = re.sub(r'\(|\)', '', loc)
    location = geolocator.reverse(loc)
    zipcode = location.raw['address']['postcode']
    return zipcode

In [ ]:
locDict = {}

In [15]:
import pyprind
import geopy

def get_zip_dict(locations):
    global locDict
    count = 0
    n = len(locations)
    bar = pyprind.ProgBar(n) 
    for i in xrange(n):
        loc = locations[i]
        if loc not in locDict:
            try:
                locDict[loc] = get_zipcode(loc)
            except geopy.exc.GeocoderServiceError as e:
                if e.message == 'HTTP Error 420: unused':
                    time.sleep(1800)
                    locDict[loc] = get_zipcode(loc)
                elif e.message == '<urlopen error [Errno 10060] A connection attempt failed because the connected party did not properly respond after a period of time, or established connection failed because connected host has failed to respond>':
                    time.sleep(10)
                    locDict[loc] = get_zipcode(loc)
        bar.update()  
locDict = get_zip_dict(sfdata['Location'].unique())

0%                          100%
[                              ]/Users/livenb/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:14: DeprecationWarning: BaseException.message has been deprecated as of Python 2.6
/Users/livenb/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:17: DeprecationWarning: BaseException.message has been deprecated as of Python 2.6
[##                            ] | ETA: 00:01:21

KeyboardInterrupt: 

In [14]:
len(locDict)

2346

In [ ]:
import pickle
with open('data/sfpost.pkl', 'wb') as f:
    pickle.dump(locDict, f, pickle.HIGHEST_PROTOCOL)

In [ ]:
sfdata['zipcode'] = sfdata['Location'].apply(lambda x: locDict[x])

In [ ]:
sfdata.to_csv(filename)

In [ ]:
sfdata.head()

In [ ]:
sfdf = sfdata.groupby(['Year', 'zipcode','CrimeCat'])['Unnamed: 0'].count()

In [ ]:
sfdf = sfdf.unstack().reset_index()

In [ ]:
sfdf = sfdf.fillna(0)

In [ ]:
from sklearn.decomposition import NMF
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

In [ ]:
sfdata['Address'].unique().shape

In [ ]:
sfdf.head()

In [ ]:
X = sfdf.drop(['Year', 'Address'], axis=1).values
year = sfdf['Year'].values

In [ ]:
X.shape

In [ ]:
from scipy.spatial.distance import cdist
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
%matplotlib inline

# scaler = StandardScaler()
# # X_trans = scaler.fit_transform(X)
# def kmeans_one_year(X, year, yr):
#     dists = []
#     scores = []
#     for k in xrange(2, 10):
#         print k
#         kmeanModel = KMeans()
#         kmeanModel.fit(X[year == yr])
#         centroids = kmeanModel.cluster_centers_
#         labels = kmeanModel.labels_
#         dist = cdist(X, centroids)
#         dists.append(dist.min(axis=1).sum())
#         score = silhouette_score(X[year == yr], labels)
#         scores.append(score)
#     fig = plt.figure(figsize=(12,8))
#     ax1 = fig.add_subplot(121)
#     ax1.plot(xrange(2, 10), dists, 'b*-')
#     ax1.grid(True)
#     ax1.set_xlabel('Number of clusters')
#     ax1.set_ylabel('sum of squares')
#     ax2 = fig.add_subplot(122)
#     ax2.plot(xrange(2, 50), scores, 'b*-')
#     ax2.grid(True)
#     ax2.set_xlabel('Number of clusters')
#     ax2.set_ylabel('slihouette score')
#     plt.show()

# kmeans_one_year(X, year, 2003)